[View in Colaboratory](https://colab.research.google.com/github/gauravsingh1012/Mining-Historical-Tweets-By-Using-Pandas-For-Data-Visualisation-and-Sentiment-Analysis/blob/master/Mastering_Pandas_For_Finance.ipynb)

#Chapter 1: Reshaping, Reorganizing and Aggregating



> ** A. Loading historical stock data from the quandl **



In [2]:
!pip install quandl

    100% |████████████████████████████████| 51kB 5.3MB/s 
  Running setup.py bdist_wheel for inflection ... - done
  Stored in directory: /content/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


In [0]:
import quandl
import pandas as pd
import numpy as np
import time

In [0]:
quandl.ApiConfig.api_key = input('Please enter your quandl Key: ')
exchange = 'WIKI'

In [3]:
ticker = 'MSFT'
msft = quandl.get('%s/%s' % (exchange, ticker))
msft.head(1)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1986-03-13,25.5,29.25,25.5,28.0,3582600.0,0.0,1.0,0.058941,0.067609,0.058941,0.06472,1.031789e+09


In [4]:
ticker = 'AAPL'
appl = quandl.get('%s/%s' % (exchange, ticker))
appl.head(1)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.42447,0.422706,0.422706,117258400.0


In [0]:
# Set the index to a column called Date
msft = msft.reset_index(level=0)
appl = appl.reset_index(level=0)




> **B. Concatenating multiple DataFrame objects**



> To demonstrate various forms of concatenation, we will start with the following
data that shows the adjusted closing prices for MSFT for the months of January and
February 2012 represented in the following command. This dataset simulates the
retrieval of stock information representing two different time periods and stores the
data in two different DataFrame objects, as shown here:






In [0]:
msft.set_index('Date', inplace = True)
appl.set_index('Date', inplace = True)

In [14]:
msftA01 = msft['2012-01'][['Adj. Close']]
msftA02 = msft['2012-02'][['Adj. Close']]
msftA01[:3]

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-04,23.332995
2012-01-05,23.571435


In [15]:
msftA02[:3]

,Adj. Close
Date,
2012-02-01,25.453403
2012-02-02,25.504497
2012-02-03,25.751452


In [16]:
pd.concat([msftA01.head(3), msftA02.head(3)])

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-04,23.332995
2012-01-05,23.571435
2012-02-01,25.453403
2012-02-02,25.504497
2012-02-03,25.751452




> The resulting DataFrame contains an index identical in structure to both of the
objects, with labels from the first object and then the second object copied into the
new object. At first glance, it may appear that the concatenation is a pure copy of the
rows from each DataFrame into the new DataFrame , but as we will see, the process is
more elaborate (and hence flexible). This will become more evident as we take a look
at more examples.
The following example concatenates the first five adjusted close values in January
for both MSFT and AAPL. These have identical index labels and result in duplicate
index labels in the new DataFrame . During a concatenation along the row axis,
pandas will not align the index labels. They will be copied and this can create
duplicate, identical index labels:

In [30]:
applA01 = appl['2012-01'][['Adj. Close']]
withDups = pd.concat([msftA01[:3], applA01[:3]])
withDups

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-04,23.332995
2012-01-05,23.571435
2012-01-03,52.848787
2012-01-04,53.132802
2012-01-05,53.722681


In [33]:
withDups.loc['2012-01-03']

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-03,52.848787




> This concatenation has lost whether the Adj Close value in the new DataFrame
came from the MSFT or AAPL DataFrame . This source DataFrame of each row
can be preserved during concatenation by specifying the value of the keys in the
new DataFrame . These keys will add an additional level to the index (making a
MultiIndex ), which then can be used to identify the source DataFrame :

In [34]:
closes = pd.concat([msftA01[:3], applA01[:3]],
keys=['MSFT', 'APPL'])
closes

Adj. Close
     Date                  
MSFT 2012-01-03   22.792249
     2012-01-04   23.332995
     2012-01-05   23.571435
APPL 2012-01-03   52.848787
     2012-01-04   53.132802
     2012-01-05   53.722681



> Using this new MultiIndex , it is then possible to extract the values for either stock
from this new DataFrame by only using the index labels. The following command
does this for the MSFT entries:

In [38]:
closes.loc['MSFT'][:3]

,Adj. Close
Date,
2012-01-03,22.792249
2012-01-04,23.332995
2012-01-05,23.571435




> Concatenation along the row axis can also be performed using DataFrame objects
with multiple columns. The following command modifies the previous example to
use the Adj Close and Adj Volume columns in each DataFrame . 

In [0]:
msftAV = msft[['Adj. Close', 'Adj. Volume']]
applAV = appl[['Adj. Close', 'Adj. Volume']]

In [42]:
pd.concat([msftAV[:3], applAV[:3]])

,Adj. Close,Adj. Volume
Date,,
1986-03-13,0.064720,1.031789e+09
1986-03-14,0.067031,3.081600e+08
1986-03-17,0.068187,1.331712e+08
1980-12-12,0.422706,1.172584e+08
1980-12-15,0.400652,4.397120e+07
1980-12-16,0.371246,2.643200e+07




> The columns in the DataFrame objects in a concatenation do not have to have the
same names. The following command demonstrates a concatenation where the
aaplA DataFrame only consists of the Adj Close column, whereas the MSFT
DataFrame has both Adj Close and Volume columns:

In [45]:
applA = appl[['Adj. Close']]
pd.concat([msftAV[:3], applA[:3]])

,Adj. Close,Adj. Volume
Date,,
1986-03-13,0.064720,1.031789e+09
1986-03-14,0.067031,3.081600e+08
1986-03-17,0.068187,1.331712e+08
1980-12-12,0.422706,NaN
1980-12-15,0.400652,NaN
1980-12-16,0.371246,NaN




> Since the rows originating from the appl DataFrame do not have a Volume column,
pandas inserts NaN into the Volume column for those rows.
The set of columns that results from a concatenation along the row axis is the result
of relational algebra across the names of the columns. In this default scenario, the
resulting column is the union of column names from each DataFrame . This can be
changed to an intersection using the join parameter. The following command makes
the set of resulting columns the intersection of the column names by specifying
join='inner' :

In [46]:
pd.concat([msftAV[:3], applA[:3]], join='inner')

,Adj. Close
Date,
1986-03-13,0.064720
1986-03-14,0.067031
1986-03-17,0.068187
1980-12-12,0.422706
1980-12-15,0.400652
1980-12-16,0.371246




> We can change the axis for concatenation to the columns using axis=1 :

In [52]:
msftA = msft[['Adj. Close']]
closes = pd.concat([msftA01[:3], applA01[:3]], axis=1)
closes

,Adj. Close,Adj. Close
Date,,
2012-01-03,22.792249,52.848787
2012-01-04,23.332995,53.132802
2012-01-05,23.571435,53.722681




> Note that this DataFrame has two Adj Close columns and only consists of 3 rows
(the concatenation along axis=0 has 6 ). Because of the use of axis=1 , the union of
the index labels is derived instead from the column names, and the columns are copied
one by one in an orderly manner from the DataFrame objects, including duplicates.
It is also possible to concatenate with multiple columns where the DataFrame objects
do not have the same set of index labels. The following command concatenates the
first five msftAV values and the first three applAV values:

In [56]:
pd.concat([msftA01[:5], applA01[:3]], axis=1, keys=['MSFT', 'APPL'])

,MSFT,APPL
,Adj. Close,Adj. Close
Date,,
2012-01-03,22.792249,52.848787
2012-01-04,23.332995,53.132802
2012-01-05,23.571435,53.722681
2012-01-06,23.933352,NaN
2012-01-09,23.622529,NaN




> This results in duplicate column names, so we use the keys parameter to create
MultiIndex for the columns. Since there were row index labels that were not found
in aaplCV , pandas fills those with NaN .
Just as with concatenation along the row axis, the type of join performed by
pd.concat() can be changed using the join parameter. The following command
performs an inner join instead of an outer join, which results in the intersection of
row index labels:

In [57]:
pd.concat([msftA01[:5], applA01[:3]], axis=1,join='inner', keys=['MSFT', 'AAPL'])

,MSFT,AAPL
,Adj. Close,Adj. Close
Date,,
2012-01-03,22.792249,52.848787
2012-01-04,23.332995,53.132802
2012-01-05,23.571435,53.722681




> The resulting DataFrame only has three rows because those index labels were the
only ones in common in the two concatenated DataFrame objects.
If you want to ignore indices in the result of pd.concat() , you can use the ignore_
index=True parameter, which will drop the index and create a default zero-based
integer index, as shown here:

In [58]:
pd.concat([msftA01[:5], applA01[:3]], ignore_index=True)

,Adj. Close
0,22.792249
1,23.332995
2,23.571435
3,23.933352
4,23.622529
5,52.848787
6,53.132802
7,53.722681




> **C. Merging DataFrame objects**


> The combination of pandas objects is allowed using relational database-like join
operations, high-performance in-memory operations, and the pd.merge() function.
Merging in pandas differs from concatenation in that the pd.merge() function
combines data based on the values of the data in one or more columns instead of
using the index label values along a specific axis.
The default process that pd.merge() uses is to first identify the columns the data of
which will be used in the merge, and then to perform an inner join based upon that
information. The columns used in the join are, by default, selected as those in both
DataFrame objects with common names (an intersection of the column labels).
To demonstrate a merge, we will use the following two DataFrame objects, one with
the volumes and the other with the adjusted close values for MSFT. Both have the
index reset:



In [0]:
msftVR = msft[['Adj. Volume']].reset_index()
msftAR = msft[['Adj. Close']].reset_index()


In [61]:
msftVR[:3]

,Date,Adj. Volume
0,1986-03-13,1.031789e+09
1,1986-03-14,3.081600e+08
2,1986-03-17,1.331712e+08


In [62]:
msftAR[:3]

,Date,Adj. Close
0,1986-03-13,0.064720
1,1986-03-14,0.067031
2,1986-03-17,0.068187


In [65]:
msftCVR = pd.merge(msftAR, msftVR)
msftCVR[:5]

,Date,Adj. Close,Adj. Volume
0,1986-03-13,0.064720,1.031789e+09
1,1986-03-14,0.067031,3.081600e+08
2,1986-03-17,0.068187,1.331712e+08
3,1986-03-18,0.066454,6.776640e+07
4,1986-03-19,0.065298,4.789440e+07




> Since both msftAR, msftVR have their indexes reset to level=0. Instead of using Date as the index, these have Date as a column so that it can be used
in the merge. Our goal is to create a DataFrame that contains a Date column and
both AdjClose and AdjVolume columns. This has been accomplished with the above statement. 


> The column in common is Date ; therefore, pandas performs an inner join on the
values in that column across both DataFrame objects. Once that set is calculated,
pandas copies in the appropriate values for each row from both DataFrame objects.
The types of joins supported by pd.merge() are similar to the different types of joins
supported in relational databases. They are as follows:



> 1.   left : Use keys from the left DataFrame (equivalent to SQL's left-outer join)
2.   right : Use keys from the right DataFrame (equivalent to SQL's right-outer
join)
3.   outer : Use the union of keys from both DataFrame objects (equivalent to
SQL's full outer join)
4.    inner : Use the intersection of keys from both DataFrame objects (equivalent
to SQL's inner join)



> To demonstrate each difference in the results between inner and outer joins, we will
use the following data:




In [0]:
msftAR0_5 = msftAR[0:5]
msftVR2_4 = msftVR[2:4]

In [67]:
msftAR0_5

,Date,Adj. Close
0,1986-03-13,0.064720
1,1986-03-14,0.067031
2,1986-03-17,0.068187
3,1986-03-18,0.066454
4,1986-03-19,0.065298


In [68]:
msftVR2_4

,Date,Adj. Volume
2,1986-03-17,133171200.0
3,1986-03-18,67766400.0


In [69]:
pd.merge(msftAR0_5, msftVR2_4)

,Date,Adj. Close,Adj. Volume
0,1986-03-17,0.068187,133171200.0
1,1986-03-18,0.066454,67766400.0




> as shown above, For an inner join, since there are only two rows with matching dates, the result only
has two rows and merges the DataFrame objects where Date values are in common.


>   This can be changed to an outer join with how='outer' . All rows from the outer
DataFrame are returned ( msftAR0_5 ), and values not found in the inner DataFrame
( msftVR2_4 ) are replaced with NaN :



In [70]:
pd.merge(msftAR0_5, msftVR2_4, how='outer')

,Date,Adj. Close,Adj. Volume
0,1986-03-13,0.064720,NaN
1,1986-03-14,0.067031,NaN
2,1986-03-17,0.068187,133171200.0
3,1986-03-18,0.066454,67766400.0
4,1986-03-19,0.065298,NaN




> **D. Pivoting**


> Financial data is often stored in a format where the data is not normalized and,
therefore, has repeated values in many columns or values that logically should exist
in other tables. An example of this would be the following, where the historical
prices for multiple stocks are represented in a single DataFrame using a Symbol
column. The following command creates a DataFrame with this schema and
populates the records:


